In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
# Kết nối với Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Định nghĩa kích thước hình ảnh
img_height, img_width = 96, 96

# Đường dẫn đến dữ liệu
data_paths = [
    "/content/drive/My Drive/Real",
    "/content/drive/My Drive/Altered-Easy",
]

# Hàm tiền xử lý hình ảnh
def preprocess_images(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_width, img_height))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)  # Thêm chiều kênh
    return img

# Hàm tải dữ liệu
def load_dataset(data_paths):
    images = []
    labels = []
    for path in data_paths:
        for file in os.listdir(path):
            if file.endswith(".BMP"):
                img_path = os.path.join(path, file)
                img = preprocess_images(img_path)
                images.append(img)
                label = int(file.split("__")[0]) - 1
                labels.append(label)
    return np.array(images), np.array(labels)

# Hàm tạo cặp hình ảnh bằng generator
def create_pairs_generator(images, labels, limit_per_class=100):
    num_classes = np.unique(labels).shape[0]
    pairs = []
    pair_labels = []

    for class_index in range(num_classes):
        class_images = images[labels == class_index]

        if len(class_images) < 2:
            continue  # Bỏ qua lớp nếu không đủ hình ảnh

        # Tạo cặp dương
        for i in range(len(class_images)):
            for j in range(i + 1, len(class_images)):
                if len(pairs) < limit_per_class:
                    pairs.append([class_images[i], class_images[j]])
                    pair_labels.append(1)  # Cặp dương
                else:
                    break  # Dừng nếu đã đạt giới hạn

        # Tạo cặp âm
        for _ in range(len(class_images)):
            if len(pairs) < limit_per_class:
                random_class_index = np.random.choice([c for c in range(num_classes) if c != class_index])
                random_image = images[labels == random_class_index]
                if len(random_image) > 0:
                    random_image = random_image[np.random.randint(len(random_image))]
                    pairs.append([class_images[i], random_image])
                    pair_labels.append(0)  # Cặp âm
            else:
                break  # Dừng nếu đã đạt giới hạn

    return np.array(pairs), np.array(pair_labels)

# Tải dataset
images, labels = load_dataset(data_paths)

# Mã hóa nhãn
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
print("kkkk")
# Tạo cặp hình ảnh
pairs, pair_labels = create_pairs_generator(images, labels_encoded, limit_per_class=1000)  # Giới hạn số lượng cặp
def create_verification_model(input_shape=(img_height, img_width, 1), embedding_size=128):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(embedding_size, activation='relu'),
        Dropout(0.5),  # Thêm Dropout trước khi đầu ra
    ])
    return model

# Tạo mô hình
verification_model = create_verification_model()

# Hàm mất mát đối kháng
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    square_pred = tf.square(y_pred)
    square_margin = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * square_margin)

# Biên dịch mô hình
verification_model.compile(optimizer=Adam(learning_rate=0.0001), loss=contrastive_loss)

# Thiết lập callback để dừng sớm và lưu trọng số mô hình tốt nhất
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    filepath='/content/drive/My Drive/fingerprint_verification_weights.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

# Huấn luyện mô hình
verification_model.fit(
    [pairs[:, 0], pairs[:, 1]],  # Cặp hình ảnh
    pair_labels,  # Nhãn cho các cặp
    epochs=100,
    batch_size=32,
    validation_split=0.2,  # Tách 20% dữ liệu cho validation
    callbacks=[early_stopping, model_checkpoint]  # Sử dụng callback
)

# Lưu mô hình đã huấn luyện
verification_model.save('/content/drive/My Drive/fingerprint_verification_model.h5')

# Kiểm tra độ chính xác của mô hình trên tập kiểm tra (nếu có)
# Bạn có thể thêm đoạn mã này khi bạn có tập kiểm tra
# test_pairs, test_pair_labels = create_pairs(test_images, test_labels)  # tạo cặp cho tập kiểm tra
# test_loss = verification_model.evaluate([test_pairs[:, 0], test_pairs[:, 1]], test_pair_labels)
# print(f'Test loss: {test_loss}')